In [12]:
import numpy as np
import pandas as pd
import os
import glob
import shutil
import tqdm
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from dataset_tools import OwnDataset, MNIST
import matplotlib.pyplot as plt

from plotting import show_img


In [5]:
data_path = r"C:\Users\Otto\Desktop\Fagprojekt_data\singh_cp_pipeline_singlecell_images"

folder_paths = glob.glob(os.path.join(data_path, "*"))

print(len(folder_paths))

# folder_paths

subset_folder_paths = folder_paths[:50]


3848


In [4]:
# for folder_path in subset_folder_paths:
#     folder_name = folder_path.split("\\")[-1]
#     new_folder_path = os.path.join(new_data_path, folder_name)
#     shutil.copytree(folder_path, new_folder_path)
#     print(f"Copying {folder_name} to {new_folder_path}")

In [13]:
# load metadata
df_metadata = pd.read_parquet("metadata.parquet")


# npy_paths = glob.glob(os.path.join(data_path, "*", "*.npy"))
npy_paths = glob.glob(os.path.join(data_path, "*", "*.npy"))
len(npy_paths)



7774

In [14]:
# use tqdm to see progress
save_path = "data_subset\\"

# get the moa for each file, and put it in a folder named after the moa
for i, npy_path in enumerate(tqdm.tqdm(npy_paths)):
    data = np.load(npy_path)
    # data = (data / np.max(data) * 255).astype(np.int8)
    lookup_path = npy_path.split("\\")[-1]
    moa = df_metadata[df_metadata["Single_Cell_Image_Name"] == lookup_path]["moa"].values[0]

    # get file under data_folder named moa
    # if not exists, create it
    os.makedirs(os.path.join(save_path, moa), exist_ok=True)
    # save the file
    file_path = os.path.join(save_path, moa, lookup_path)
    np.save(file_path, data)

    
    # if i == 0:
    #     total_data = np.expand_dims(data, axis=0)
    # else:
    #     total_data = np.vstack((total_data, np.expand_dims(data, axis=0)))
    # if i == 1000:
    #     break

100%|██████████| 7774/7774 [03:02<00:00, 42.57it/s]


In [ ]:
dataset = MNIST(train=True, transform=transforms.ToTensor())
dataset

In [ ]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
x, y = next(iter(loader))

y

In [ ]:

def npy_loader(path):
    """
    load a npy file and change the dtype to int8
    """
    sample = np.load(path)

    sample = (sample / np.max(sample) * 255).astype(np.uint8)

    sample = torch.from_numpy(sample)
    # make channel the first dimension
    sample = sample.permute(2, 0, 1)

    return sample


In [ ]:
dataset = datasets.DatasetFolder("data_subset/singh_cp_pipeline_singlecell_images", loader=npy_loader, extensions=('.npy',))
# dataset = OwnDataset()

dataset[3050][0].shape
show_img(dataset[3051][0])

In [ ]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# metadata

In [ ]:
# df_metadata = pd.read_csv("metadata.csv", index_col=0)
# df_features = pd.read_csv("CPfeatures.csv", index_col=0) # very large file (4.2 GB)

# read from parquet instead of csv

# df_features = pd.read_parquet("CPfeatures.parquet")

In [ ]:
df_metadata.head()
# df_features.head()

In [ ]:
df_metadata["Multi_Cell_Image_Id"].unique()